In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os 
import cv2 as cv 
import glob
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler,Subset
from torchvision import transforms
from IPython.display import HTML
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torchvision  import datasets 
from sklearn.model_selection import KFold
import optuna
from optuna.trial import Trial
from tqdm import tqdm
import torch.optim as optim
import torch.nn as nn
from torchvision import models
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Subset, DataLoader

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
combined_data = "/kaggle/input/deepfake-and-real-images/Dataset/Test"

In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.ToTensor(),         # Convert to Tensor
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])  # Normalize
])

In [6]:
combinedData = datasets.ImageFolder(combined_data, transform=transform)

In [7]:
combinedData

Dataset ImageFolder
    Number of datapoints: 10905
    Root location: /kaggle/input/deepfake-and-real-images/Dataset/Test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [8]:
kf = KFold(n_splits=3, shuffle=True, random_state=42) 

In [9]:
samples = len(combinedData)
samples

10905

In [10]:
folds = list(kf.split(np.arange(samples)))

In [11]:
folds

[(array([    1,     2,     4, ..., 10900, 10901, 10902]),
  array([    0,     3,     8, ..., 10899, 10903, 10904])),
 (array([    0,     1,     3, ..., 10902, 10903, 10904]),
  array([    2,    15,    18, ..., 10889, 10900, 10901])),
 (array([    0,     2,     3, ..., 10901, 10903, 10904]),
  array([    1,     4,     5, ..., 10896, 10897, 10902]))]

# Split our data into 3 Kfolds

In [12]:
for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
    print(f"Fold {fold_idx + 1}")
    
    train_size = int(0.75* len(train_val_idx)) 
    val_size = len(train_val_idx) - train_size #25
    
    train_idx = train_val_idx[:train_size]
    val_idx = train_val_idx[train_size:]
    
    # Create subsets
    train_set = Subset(combinedData, train_idx)
    val_set = Subset(combinedData, val_idx)
    test_set = Subset(combinedData, test_idx)
    
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
    
    print(f"Train size: {len(train_set)}")
    print(f"Validation size: {len(val_set)}")
    print(f"Test size: {len(test_set)}")

Fold 1
Train size: 5452
Validation size: 1818
Test size: 3635
Fold 2
Train size: 5452
Validation size: 1818
Test size: 3635
Fold 3
Train size: 5452
Validation size: 1818
Test size: 3635


# Define our model mobilenet

In [12]:
def get_model(name):
    if name == "densenet121":
        # Load the pretrained DenseNet121 model
        model = models.densenet121(pretrained=True)
        
        # Freeze all layers
        for param in model.parameters():
            param.requires_grad = False
        
        # Modify the classifier for the desired number of output classes
        num_features = model.classifier.in_features
        model.classifier = nn.Linear(num_features, 2)
        
        # Print the number of trainable parameters (should be only the classifier)
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Total trainable parameters: {trainable_params:,}")
    
    else:
        raise ValueError("Model name must be 'densenet121'")
    
    return model


# Training function

In [13]:
def TrainingModels(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)  #loss function used to compute the error between predictions and true labels
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1) # max score from the scores of two classes
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc




# Eval function

In [14]:
def evaluate(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

# Optimize the Learning rate using Optuna

In [16]:
def LR_optimization(trial: Trial):
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
    
    # Iterate over folds
    fold_results = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"\nFold {fold_idx + 1}")
        
        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

        
        for model_name in ["densenet121"]:
            print(f"\nTraining model: {model_name.upper()} on Fold {fold_idx + 1}")
            
            # Initialize model
            model = get_model(model_name).to(device)
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            for epoch in range(5):
                current_lr = optimizer.param_groups[0]["lr"]
                print(f"\nEpoch {epoch + 1}/5 | Learning Rate: {current_lr:.6f}")
                
                # Train and evaluate
                train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
                val_loss, val_acc = evaluate(model, val_loader, criterion)

                print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
                print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
            _, val_acc = evaluate(model, val_loader, criterion)
            fold_results.append(val_acc) 

    mean_val_acc = sum(fold_results) / len(fold_results)
    return mean_val_acc



## Without Fine-tune 

In [17]:
study = optuna.create_study(direction="maximize")
study.optimize(LR_optimization, n_trials=5)

best_trial = study.best_trial
print("\nBest Trial:")
print(f"  Value (Mean Validation Accuracy): {best_trial.value}")
print(f"  Params: {best_trial.params}")

[I 2024-12-24 23:13:10,156] A new study created in memory with name: no-name-be131352-c94a-438d-ad7c-418862414302
<ipython-input-16-3906ab1e677d>:2: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-5, 1e-2)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-da


Fold 1

Training model: DENSENET121 on Fold 1


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 168MB/s]


Total trainable parameters: 2,050

Epoch 1/5 | Learning Rate: 0.000013


Train Loss: 0.6601, Train Acc: 0.6328
Val Loss: 1.0801, Val Acc: 0.0776

Epoch 2/5 | Learning Rate: 0.000013


...

Train Loss: 0.5048, Train Acc: 0.7405
Val Loss: 0.9120, Val Acc: 0.3971

Epoch 4/5 | Learning Rate: 0.000545


Train Loss: 0.4862, Train Acc: 0.7594
Val Loss: 0.9517, Val Acc: 0.3872

Epoch 5/5 | Learning Rate: 0.000545


Train Loss: 0.4749, Train Acc: 0.7639
Val Loss: 0.8540, Val Acc: 0.4692


[I 2024-12-25 00:09:57,418] Trial 4 finished with value: 0.47048038137147047 and parameters: {'lr': 0.0005446615699330324}. Best is trial 4 with value: 0.47048038137147047.



Best Trial:
  Value (Mean Validation Accuracy): 0.47048038137147047
  Params: {'lr': 0.0005446615699330324}


# After the Optuna optimization process, we will use the best trial to set the learning rate

In [18]:
# Best learning rate from Optuna
best_lr = 0.0005446615699330324

In [20]:
final_results = {}

for model_name in ["densenet121"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    # Initialize model
    model = get_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    
    # Create DataLoaders (assuming combinedData is split into train and test datasets)
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

    # Train the model
    for epoch in range(10):  # Example: 10 epochs
        print(f"\nEpoch {epoch + 1}/10")
        
        # Train and evaluate
        train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    
    # Final evaluation on the test set
    test_loss, test_acc = evaluate(model, test_loader, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    
    # Store results
    final_results[model_name] = {
        "Test Loss": test_loss,
        "Test Accuracy": test_acc
    }

# Print final results
print("\nFinal Results:")
for model_name, metrics in final_results.items():
    print(f"{model_name.upper()}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")


Training model: DENSENET121 with best learning rate 0.0005446615699330324
Total trainable parameters: 2,050

Epoch 1/10


Train Loss: 0.5733, Train Acc: 0.7040

Epoch 2/10


Train Loss: 0.5172, Train Acc: 0.7368

Epoch 3/10


Train Loss: 0.4975, Train Acc: 0.7550

Epoch 4/10


Train Loss: 0.4865, Train Acc: 0.7603

Epoch 5/10


...

Train Loss: 0.4624, Train Acc: 0.7729

Epoch 8/10


Train Loss: 0.4627, Train Acc: 0.7784

Epoch 9/10


Train Loss: 0.4470, Train Acc: 0.7900

Epoch 10/10


Train Loss: 0.4437, Train Acc: 0.7863


Test Loss: 0.5409, Test Acc: 0.7114

Final Results:
DENSENET121:
  Test Loss: 0.5409
  Test Accuracy: 0.7114


In [21]:
# Function to evaluate the model and get performance metrics
def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

# Initialize cross-validation metrics storage
confusion_matrices = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

best_lr = 0.0005446615699330324

for model_name in ["densenet121"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"Fold {fold_idx + 1}")

        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
        
        
        model = get_model(model_name).to(device)
        optimizer = optim.Adam(model.parameters(), lr=best_lr)
        criterion = nn.CrossEntropyLoss()

        # Train the model
        for epoch in range(10):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(f"Epoch {epoch+1}/10, Loss: {running_loss / len(train_loader):.4f}")

    
        preds, labels = evaluate_model(model, test_loader)

        
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average='weighted', zero_division=0)
        recall = recall_score(labels, preds, average='weighted', zero_division=0)
        f1 = f1_score(labels, preds, average='weighted', zero_division=0)
        cm = confusion_matrix(labels, preds)

        # Store metrics for the fold
        confusion_matrices.append(cm)
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f"Fold {fold_idx + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        print(f"Confusion Matrix:\n{cm}\n")

    # Average metrics across folds
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1_score = np.mean(f1_scores)
    avg_confusion_matrix = np.mean(confusion_matrices, axis=0)

    # Print final average metrics for the model
    print(f"\nFinal Results for {model_name.upper()}:")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1-Score: {avg_f1_score:.4f}")
    print(f"Average Confusion Matrix:\n{avg_confusion_matrix}")


Training model: DENSENET121 with best learning rate 0.0005446615699330324
Fold 1
Total trainable parameters: 2,050
Epoch 1/10, Loss: 0.5823
Epoch 2/10, Loss: 0.5212
Epoch 3/10, Loss: 0.5043
Epoch 4/10, Loss: 0.4821
Epoch 5/10, Loss: 0.4778
Epoch 6/10, Loss: 0.4746
Epoch 7/10, Loss: 0.4670
Epoch 8/10, Loss: 0.4586
Epoch 9/10, Loss: 0.4575
Epoch 10/10, Loss: 0.4511
Fold 1 - Accuracy: 0.7004, Precision: 0.7223, Recall: 0.7004, F1-Score: 0.6907
Confusion Matrix:
[[1618  246]
 [ 843  928]]

Fold 2
Total trainable parameters: 2,050


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.5737
Epoch 2/10, Loss: 0.5114
Epoch 3/10, Loss: 0.4935
Epoch 4/10, Loss: 0.4799
Epoch 5/10, Loss: 0.4688
Epoch 6/10, Loss: 0.4638
Epoch 7/10, Loss: 0.4580
Epoch 8/10, Loss: 0.4474
Epoch 9/10, Loss: 0.4507
Epoch 10/10, Loss: 0.4444
Fold 2 - Accuracy: 0.6894, Precision: 0.7208, Recall: 0.6894, F1-Score: 0.6770
Confusion Matrix:
[[1617  217]
 [ 912  889]]

Fold 3
Total trainable parameters: 2,050


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.5868
Epoch 2/10, Loss: 0.5229
Epoch 3/10, Loss: 0.4980
Epoch 4/10, Loss: 0.4860
Epoch 5/10, Loss: 0.4726
Epoch 6/10, Loss: 0.4720
Epoch 7/10, Loss: 0.4710
Epoch 8/10, Loss: 0.4616
Epoch 9/10, Loss: 0.4520
Epoch 10/10, Loss: 0.4486
Fold 3 - Accuracy: 0.6897, Precision: 0.7215, Recall: 0.6897, F1-Score: 0.6794
Confusion Matrix:
[[1568  226]
 [ 902  939]]


Final Results for DENSENET121:
Average Accuracy: 0.6932
Average Precision: 0.7215
Average Recall: 0.6932
Average F1-Score: 0.6824
Average Confusion Matrix:
[[1601.          229.66666667]
 [ 885.66666667  918.66666667]]


## Fine-tuning 

## Applying Fine-tune

In [13]:
def get_model(name):
    if name == "densenet121":
        # Load the pretrained DenseNet121 model
        model = models.densenet121(pretrained=True)
        
        # Freeze all layers initially
        for param in model.parameters():
            param.requires_grad = False
        
        # Fine-tune the last 5 layers:
        # Unfreeze the last 5 dense blocks and transition layers
        for param in model.features[-5:].parameters():
            param.requires_grad = True
        
        # Modify the classifier for the desired number of output classes
        num_features = model.classifier.in_features
        model.classifier = nn.Linear(num_features, 2)
        
        # Ensure the classifier is trainable
        for param in model.classifier.parameters():
            param.requires_grad = True
        
        # Print the number of trainable parameters
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f"Total trainable parameters: {trainable_params:,}")
    
    else:
        raise ValueError("Model name must be 'densenet121'")
    
    return model


## Training Function 

In [14]:
# Fine-tune

def TrainingModels(model, train_loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    # Add gradient clipping to prevent exploding gradients
    max_grad_norm = 1.0
    
    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        
        # Clip gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [15]:
def evaluate(model, data_loader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in tqdm(data_loader, desc="Evaluating", leave=False):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc

In [16]:
# Fine-tune

def LR_optimization(trial: Trial):
    # Use a smaller learning rate range for fine-tuning
    lr = trial.suggest_loguniform("lr", 1e-6, 1e-3)
    
    # Rest of the function remains the same
    fold_results = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"\nFold {fold_idx + 1}")
        
        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

        for model_name in ["densenet121"]:
            print(f"\nFine-tuning model: {model_name.upper()} on Fold {fold_idx + 1}")
            
            model = get_model(model_name).to(device)
            criterion = nn.CrossEntropyLoss()
            
            # Use a smaller weight decay for fine-tuning
            optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
            
            # Add learning rate scheduler for fine-tuning
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode='max', factor=0.5, patience=2, verbose=True
            )

            for epoch in range(5):
                current_lr = optimizer.param_groups[0]["lr"]
                print(f"\nEpoch {epoch + 1}/5 | Learning Rate: {current_lr:.6f}")
                
                train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
                val_loss, val_acc = evaluate(model, val_loader, criterion)
                
                # Update learning rate based on validation accuracy
                scheduler.step(val_acc)

                print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
                print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
            
            _, val_acc = evaluate(model, val_loader, criterion)
            fold_results.append(val_acc)

    mean_val_acc = sum(fold_results) / len(fold_results)
    return mean_val_acc

In [17]:
study = optuna.create_study(direction="maximize")
study.optimize(LR_optimization, n_trials=8)

best_trial = study.best_trial
print("\nBest Trial:")
print(f"  Value (Mean Validation Accuracy): {best_trial.value}")
print(f"  Params: {best_trial.params}")

[I 2024-12-25 13:40:56,638] A new study created in memory with name: no-name-ddf37ecc-740e-4749-83c4-30d2cad80b3f
<ipython-input-16-18e5c9fed5ec>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform("lr", 1e-6, 1e-3)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-da


Fold 1

Fine-tuning model: DENSENET121 on Fold 1


100%|██████████| 30.8M/30.8M [00:00<00:00, 156MB/s] 


Total trainable parameters: 5,658,370


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


...

Train Loss: 0.1677, Train Acc: 0.9349
Val Loss: 0.4039, Val Acc: 0.7943

Epoch 4/5 | Learning Rate: 0.000018


Train Loss: 0.1104, Train Acc: 0.9607
Val Loss: 0.5003, Val Acc: 0.7800

Epoch 5/5 | Learning Rate: 0.000018


Train Loss: 0.0653, Train Acc: 0.9791
Val Loss: 0.4024, Val Acc: 0.8339


[I 2024-12-25 15:35:10,034] Trial 7 finished with value: 0.8089475614228089 and parameters: {'lr': 1.7624322855256126e-05}. Best is trial 1 with value: 0.8912724605793914.



Best Trial:
  Value (Mean Validation Accuracy): 0.8912724605793914
  Params: {'lr': 0.0008263872006727257}


# After the Optuna optimization process, we will use the best trial to set the learning rate

In [18]:
# Best learning rate from Optuna
best_lr = 0.0008263872006727257

In [20]:
final_results = {}

for model_name in ["densenet121"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    # Initialize model
    model = get_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    
    # Create DataLoaders (assuming combinedData is split into train and test datasets)
    train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_set, batch_size=32, shuffle=False)

    # Train the model
    for epoch in range(10):  # Example: 10 epochs
        print(f"\nEpoch {epoch + 1}/10")
        
        # Train and evaluate
        train_loss, train_acc = TrainingModels(model, train_loader, criterion, optimizer)
        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    
    # Final evaluation on the test set
    test_loss, test_acc = evaluate(model, test_loader, criterion)
    print(f"Test Loss: {test_loss:.4f}, Test Acc: {test_acc:.4f}")
    
    # Store results
    final_results[model_name] = {
        "Test Loss": test_loss,
        "Test Accuracy": test_acc
    }

# Print final results
print("\nFinal Results:")
for model_name, metrics in final_results.items():
    print(f"{model_name.upper()}:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")


Training model: DENSENET121 with best learning rate 0.0008263872006727257
Total trainable parameters: 5,658,370

Epoch 1/10


Train Loss: 0.3183, Train Acc: 0.8597

Epoch 2/10


Train Loss: 0.1754, Train Acc: 0.9292

Epoch 3/10


Train Loss: 0.1074, Train Acc: 0.9596

Epoch 4/10


Train Loss: 0.0741, Train Acc: 0.9727

Epoch 5/10


...

Train Loss: 0.0288, Train Acc: 0.9894

Epoch 8/10


Train Loss: 0.0364, Train Acc: 0.9888

Epoch 9/10


Train Loss: 0.0203, Train Acc: 0.9932

Epoch 10/10


Train Loss: 0.0238, Train Acc: 0.9921


Test Loss: 0.3673, Test Acc: 0.9224

Final Results:
DENSENET121:
  Test Loss: 0.3673
  Test Accuracy: 0.9224


In [21]:
# Function to evaluate the model and get performance metrics
def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    return np.array(all_preds), np.array(all_labels)

# Initialize cross-validation metrics storage
confusion_matrices = []
accuracies = []
precisions = []
recalls = []
f1_scores = []

best_lr = 0.0008263872006727257

for model_name in ["densenet121"]:
    print(f"\nTraining model: {model_name.upper()} with best learning rate {best_lr}")
    
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(np.arange(samples))):
        print(f"Fold {fold_idx + 1}")

        train_size = int(0.75 * len(train_val_idx))
        train_idx = train_val_idx[:train_size]
        val_idx = train_val_idx[train_size:]

        train_set = Subset(combinedData, train_idx)
        val_set = Subset(combinedData, val_idx)
        test_set = Subset(combinedData, test_idx)

        train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=32, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=32, shuffle=False)
        
        
        model = get_model(model_name).to(device)
        optimizer = optim.Adam(model.parameters(), lr=best_lr)
        criterion = nn.CrossEntropyLoss()

        # Train the model
        for epoch in range(10):
            model.train()
            running_loss = 0.0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

            print(f"Epoch {epoch+1}/10, Loss: {running_loss / len(train_loader):.4f}")

    
        preds, labels = evaluate_model(model, test_loader)

        
        accuracy = accuracy_score(labels, preds)
        precision = precision_score(labels, preds, average='weighted', zero_division=0)
        recall = recall_score(labels, preds, average='weighted', zero_division=0)
        f1 = f1_score(labels, preds, average='weighted', zero_division=0)
        cm = confusion_matrix(labels, preds)

        # Store metrics for the fold
        confusion_matrices.append(cm)
        accuracies.append(accuracy)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

        print(f"Fold {fold_idx + 1} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")
        print(f"Confusion Matrix:\n{cm}\n")

    # Average metrics across folds
    avg_accuracy = np.mean(accuracies)
    avg_precision = np.mean(precisions)
    avg_recall = np.mean(recalls)
    avg_f1_score = np.mean(f1_scores)
    avg_confusion_matrix = np.mean(confusion_matrices, axis=0)

    # Print final average metrics for the model
    print(f"\nFinal Results for {model_name.upper()}:")
    print(f"Average Accuracy: {avg_accuracy:.4f}")
    print(f"Average Precision: {avg_precision:.4f}")
    print(f"Average Recall: {avg_recall:.4f}")
    print(f"Average F1-Score: {avg_f1_score:.4f}")
    print(f"Average Confusion Matrix:\n{avg_confusion_matrix}")


Training model: DENSENET121 with best learning rate 0.0008263872006727257
Fold 1
Total trainable parameters: 5,658,370
Epoch 1/10, Loss: 0.3023
Epoch 2/10, Loss: 0.1586
Epoch 3/10, Loss: 0.0971
Epoch 4/10, Loss: 0.0824
Epoch 5/10, Loss: 0.0542
Epoch 6/10, Loss: 0.0532
Epoch 7/10, Loss: 0.0358
Epoch 8/10, Loss: 0.0386
Epoch 9/10, Loss: 0.0292
Epoch 10/10, Loss: 0.0277
Fold 1 - Accuracy: 0.9208, Precision: 0.9254, Recall: 0.9208, F1-Score: 0.9204
Confusion Matrix:
[[1817   47]
 [ 241 1530]]

Fold 2
Total trainable parameters: 5,658,370


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.2996
Epoch 2/10, Loss: 0.1700
Epoch 3/10, Loss: 0.0987
Epoch 4/10, Loss: 0.0681
Epoch 5/10, Loss: 0.0468
Epoch 6/10, Loss: 0.0398
Epoch 7/10, Loss: 0.0525
Epoch 8/10, Loss: 0.0227
Epoch 9/10, Loss: 0.0213
Epoch 10/10, Loss: 0.0313
Fold 2 - Accuracy: 0.9232, Precision: 0.9280, Recall: 0.9232, F1-Score: 0.9230
Confusion Matrix:
[[1791   43]
 [ 236 1565]]

Fold 3
Total trainable parameters: 5,658,370


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.2993
Epoch 2/10, Loss: 0.1568
Epoch 3/10, Loss: 0.1068
Epoch 4/10, Loss: 0.0836
Epoch 5/10, Loss: 0.0494
Epoch 6/10, Loss: 0.0582
Epoch 7/10, Loss: 0.0304
Epoch 8/10, Loss: 0.0257
Epoch 9/10, Loss: 0.0409
Epoch 10/10, Loss: 0.0224
Fold 3 - Accuracy: 0.9387, Precision: 0.9391, Recall: 0.9387, F1-Score: 0.9386
Confusion Matrix:
[[1710   84]
 [ 139 1702]]


Final Results for DENSENET121:
Average Accuracy: 0.9276
Average Precision: 0.9308
Average Recall: 0.9276
Average F1-Score: 0.9274
Average Confusion Matrix:
[[1772.66666667   58.        ]
 [ 205.33333333 1599.        ]]
